In [ ]:
#Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
columnlist = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13']
k=1 #to adjust the row indices
ClevelandHD = pd.read_csv("processed.cleveland.data",delimiter=",",names=columnlist)
ClevelandHDMat = ClevelandHD.values
  


In [3]:
#Preprocess Data

A11arr = np.where(ClevelandHDMat[:,11]=='?')
A12arr = np.where(ClevelandHDMat[:,12]=='?')

A11 = np.delete(ClevelandHDMat[:,11],A11arr)
A12 = np.delete(ClevelandHDMat[:,12],A12arr)
A11 = np.mean(A11.astype(np.float))
A12 = np.mean(A12.astype(np.float))

#Assign Random values to the missing records in "C11" ("Major Vessels Colored")
A11 = np.array(np.random.randint(0,4,4),dtype=float)
ClevelandHD.iloc[165+k,11] = str(A11[0])
ClevelandHD.iloc[191+k,11] = str(A11[1])
ClevelandHD.iloc[286+k,11] = str(A11[2])
ClevelandHD.iloc[301+k,11] = str(A11[3])

#Assign missing values in 'C12' randomly to '3' and '6'
A12 = [3.0,6.0]
ClevelandHD.iloc[86+k,12] = str(A12[0])
ClevelandHD.iloc[265+k,12] = str(A12[1])

ClevelandHDMat = ClevelandHD.values
ClevelandHDMat = ClevelandHDMat.astype(float)

#Classify groups-(1,2,3,4) into one group
for i in range(0,len(ClevelandHD)):
    if ClevelandHD.values[i,13]>=2:
        ClevelandHDMat[i,13]=1
        ClevelandHD.iloc[i,13]=1


In [4]:
import matplotlib.pyplot as plt
%matplotlib tk

# print type(ClevelandHD.iloc[1,1])

ax = ClevelandHD.iloc[0:302,13].plot.hist(color='red')
ax.set_xlabel('Disease Diagnosis',size=20,fontweight='bold')
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
ax.set_ylabel('Counts',size=20,fontweight='bold')
ax.set_title('Distribution of Disease Diagnosis',fontsize=20,fontweight='bold')



Text(0.5,1,'Distribution of Disease Diagnosis')

In [5]:
#Plotting
%matplotlib tk

# for i in range(0,14):
#     print type(ClevelandHDMat[4,i])
    
# Plot the Histogram
fig = plt.figure()
# ax = fig.add_subplot(111)
ClevelandHD.hist(sharex=False,sharey=False,xlabelsize=1,ylabelsize=1)
plt.show()

# Plot the Density_Plot
fig = plt.figure()
# ax = fig.add_subplot(111)
ClevelandHD.plot(kind='Density',subplots=True,layout=(4,4))
plt.show()

# Plot the Heat_Map
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(ClevelandHD.corr(),vmin=-1,vmax=1,interpolation='none')
fig.colorbar(cax)
plt.show()

print ClevelandHD.corr()

           C0        C1        C2        C3        C4        C5        C6  \
C0   1.000000 -0.097542  0.104139  0.284946  0.208950  0.118530  0.148868   
C1  -0.097542  1.000000  0.010084 -0.064456 -0.199915  0.047862  0.021647   
C2   0.104139  0.010084  1.000000 -0.036077  0.072319 -0.039975  0.067505   
C3   0.284946 -0.064456 -0.036077  1.000000  0.130120  0.175340  0.146560   
C4   0.208950 -0.199915  0.072319  0.130120  1.000000  0.009841  0.171043   
C5   0.118530  0.047862 -0.039975  0.175340  0.009841  1.000000  0.069564   
C6   0.148868  0.021647  0.067505  0.146560  0.171043  0.069564  1.000000   
C7  -0.393806 -0.048663 -0.334422 -0.045351 -0.003432 -0.007854 -0.083389   
C8   0.091661  0.146201  0.384060  0.064762  0.061310  0.025665  0.084867   
C9   0.203805  0.102173  0.202277  0.189171  0.046564  0.005747  0.114133   
C10  0.161770  0.037533  0.152050  0.117382 -0.004062  0.059894  0.133946   
C13  0.223120  0.276816  0.414446  0.150825  0.085164  0.025264  0.169202   

In [6]:
#Define Train and Validation Sets

X = ClevelandHDMat[:,0:12]
y = ClevelandHDMat[:,13]

validation_size = 0.2
seed = 3
X_train, X_validationset, y_train, y_validationset = train_test_split(X,y,test_size=validation_size,random_state=seed)

print len(X_train), len(X_validationset),len(X_train)+len(X_validationset),len(X)

242 61 303 303


In [7]:
#Define Models

models = []
results = []

models.append(['LR',LogisticRegression()])
models.append(['LDA',LinearDiscriminantAnalysis()])
models.append(['CART',DecisionTreeClassifier()])
models.append(['SVM',SVC()])
models.append(['KNN',KNeighborsClassifier()])
models.append(['NB',GaussianNB()])

#Define Metrics
names = []
scoring = 'accuracy'
num_folds = 10
print ""

#Evaluate Models
for name,model in models:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f %f" % (name,cv_results.mean(),cv_results.std())
    print msg

print ""



LR: 0.801333 0.114508


/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LDA: 0.797167 0.099792
CART: 0.772833 0.058860
SVM: 0.537167 0.121053
KNN: 0.620333 0.097487
NB: 0.813667 0.088440



/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [18]:
#Compare the different Algorithm Responses
fig = plt.figure()
fig.suptitle("Comparison of Algorithms",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithm',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()


In [19]:
#Standardize the Models

models_standardized = []
models_standardized.append(('ScaledLR',Pipeline([('Scaler',StandardScaler()),('LR',LogisticRegression())])))
models_standardized.append(('ScaledLDA',Pipeline([('Scaler',StandardScaler()),('LDA',LinearDiscriminantAnalysis())])))
models_standardized.append(('ScaledCART',Pipeline([('Scaler',StandardScaler()),('CART',DecisionTreeClassifier())])))
models_standardized.append(('ScaledKNN',Pipeline([('Scaler',StandardScaler()),('KNN',KNeighborsClassifier())])))
models_standardized.append(('ScaledNB',Pipeline([('Scaler',StandardScaler()),('NB',GaussianNB())])))
models_standardized.append(('ScaledSVM',Pipeline([('Scaler',StandardScaler()),('SVM',SVC())])))

results_standardized = []
names = []

for name,model in models_standardized:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,y_train,cv=kfold,scoring=scoring)
    results_standardized.append(cv_results)
    names.append(name)
    msg = "%s: %f %f %f" % (name, cv_results.mean(), cv_results.std(), cv_results.std()*100/cv_results.mean())
    print msg

ScaledLR: 0.801500 0.106650 13.306252
ScaledLDA: 0.797167 0.099792 12.518284
ScaledCART: 0.760333 0.054873 7.216911
ScaledKNN: 0.772167 0.078334 10.144638
ScaledNB: 0.813667 0.088440 10.869262
ScaledSVM: 0.817833 0.077485 9.474418


In [20]:
#Plot the Responses from Standardization

fig = plt.figure()
fig.suptitle("Comparison of Algorithms",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.boxplot(results_standardized)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithm',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()

In [23]:
#Tuning the Algorithms

#1, Tune The Scaled LR Model

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
kfold = KFold(n_splits=num_folds,random_state=seed)
c_values = [1,3,5,7,9,11,13,15,17,19,21]
param_grid = dict(C = c_values)
model = LogisticRegression()
grid = GridSearchCV(estimator=model,cv=kfold,scoring=scoring,param_grid=param_grid)
grid_result = grid.fit(rescaledX,y_train)
print ""
print "Best score is %f and the best parameters are %s" % (grid_result.best_score_,grid_result.best_params_)
print ""

mean_grid = grid_result.cv_results_['mean_test_score']
std_grid = grid_result.cv_results_['std_test_score']

fig = plt.figure()
ax = fig.add_subplot(111)
fig.suptitle("Tuned Scaled Logistic Regression Model",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.scatter(c_values,mean_grid)
plt.errorbar(c_values,mean_grid,std_grid)
ax.set_xticklabels(c_values)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('C_values',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()


Best score is 0.801653 and the best parameters are {'C': 1}



/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/krishnaveni/anaconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [24]:
#Tuning the Algorithm

#2 Tune the KNN Model

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
kfold = KFold(n_splits=num_folds,random_state=seed)
neighbors = [1,3,5,7,9,11,13,15,17,19,21]
param_grid = dict(n_neighbors=neighbors)
model = KNeighborsClassifier()
grid = GridSearchCV(estimator=model,cv=kfold,scoring=scoring,param_grid=param_grid)
grid_result = grid.fit(rescaledX,y_train)
print ""
print "Best score is %f and the best parameters are %s" % (grid_result.best_score_,grid_result.best_params_)
print ""

mean_grid = grid_result.cv_results_['mean_test_score']
std_grid = grid_result.cv_results_['std_test_score']

fig = plt.figure()
ax = fig.add_subplot(111)
fig.suptitle("Tuned Scaled Logistic Regression Model",fontsize=20,fontweight='bold')
plt.scatter(neighbors,mean_grid)
plt.errorbar(neighbors,mean_grid,std_grid)

ax.set_xticklabels(neighbors)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('k_neighbors',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()


Best score is 0.805785 and the best parameters are {'n_neighbors': 21}



/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [28]:
# TRY ENSEMBLE METHODS (K-FOLD CV)
ensembles = []
ensembles.append(('AB',AdaBoostClassifier()))
ensembles.append(('GBM',GradientBoostingClassifier()))
ensembles.append(('RF',RandomForestClassifier()))
ensembles.append(('ET',ExtraTreesClassifier()))
results = []
names = []

for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
#Compare the different ENSEMBLE Algorithm Responses
fig = plt.figure()
ax = fig.add_subplot(111)
fig.suptitle("Comparison of Ensemble Algorithms (10-Fold CV)",fontsize=20,fontweight='bold')
plt.boxplot(results)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithms',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()

AB: 0.777167 (0.099488)
GBM: 0.796833 (0.106910)
RF: 0.780833 (0.074823)
ET: 0.801000 (0.070132)


In [31]:
#1.TRY Fit-Predict using Boosting Algorithms on the Train-Validation Sets

# model = LogisticRegression()
models = []
models.append(['AB',AdaBoostClassifier()])
models.append(['GBM',GradientBoostingClassifier()])
models.append(['RF',RandomForestClassifier()])
models.append(['ET',ExtraTreesClassifier()])

names = []
results = []

for name,model in models:
    model.fit(X_train,y_train)
    result = model.predict(X_validationset)
    print name, accuracy_score(y_validationset,result)
    names.append(name)
    results.append(accuracy_score(y_validationset,result))
    
fig = plt.figure()
ax = fig.add_subplot(111)
fig.suptitle("Predict-Score Ensemble Algorithms",fontsize=20,fontweight='bold')
ax.scatter(names,results,color='blue')
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithms',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()

print ""
print ""
print names
print results

AB 0.8032786885245902
GBM 0.7704918032786885
RF 0.7213114754098361
ET 0.8032786885245902


['AB', 'GBM', 'RF', 'ET']
[0.8032786885245902, 0.7704918032786885, 0.7213114754098361, 0.8032786885245902]


In [32]:
#2.TRY Fit-Predict using Boosting Algorithms on the Scaled Train-Validation Sets

models = []
models.append(['AB',AdaBoostClassifier()])
models.append(['GBM',GradientBoostingClassifier()])
models.append(['RF',RandomForestClassifier()])
models.append(['ET',ExtraTreesClassifier()])

names = []
results = []

for name,model in models:
    scaler = StandardScaler().fit(X_train)
    rescaledX = scaler.transform(X_train)
    model.fit(rescaledX,y_train)
    rescaled_ValidationX = scaler.transform(X_validationset)
    result = model.predict(rescaled_ValidationX)
    print name, accuracy_score(y_validationset,result)
    names.append(name)
    results.append(accuracy_score(y_validationset,result))
    

fig = plt.figure()
ax = fig.add_subplot(111)
fig.suptitle("Predict-Score Scaled Ensemble Algorithms",fontsize=20,fontweight='bold')
ax.scatter(names,results,color='blue')
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithms',size=20,fontweight='bold')
plt.ylabel('Accuracy',size=20,fontweight='bold')
plt.show()

AB 0.8032786885245902
GBM 0.7704918032786885
RF 0.7704918032786885
ET 0.8032786885245902


In [ ]:
models=[]

In [34]:
#Tune the Scaled-SVM Model

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9,2.1,2.3,2.5,2.7,2.9,3.1,3.3,3.5,3.7,4.1,4.3,4.5,4.7,4.9,5.1]
kernel_values = ['linear','poly','rbf','sigmoid']
param_grid = dict(C=c_values,kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=num_folds,random_state=seed)
grid = GridSearchCV(estimator=model,scoring=scoring,cv=kfold,param_grid=param_grid)
grid_result = grid.fit(rescaledX,y_train)
print grid_result.best_score_, grid_result.best_params_

mean_test_score = grid_result.cv_results_['mean_test_score']
std_test_score = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

# print mean_test_score

c_values = np.arange(0.1,5.1,0.1)
name = 'SVM'
for c in c_values:
    model = SVC(C=c)
    scaler = StandardScaler().fit(X_train)
    rescaledX = scaler.transform(X_train)
    model.fit(rescaledX,y_train)
    rescaled_ValidationX = scaler.transform(X_validationset)
    result = model.predict(rescaled_ValidationX)
    print name,"c=",c,":", accuracy_score(y_validationset,result)

    
# fig = plt.figure()
# ax = fig.add_subplot(111)
# fig.suptitle("Tuned Scaled Logistic Regression Model",fontsize=20,fontweight='bold')
# plt.scatter(neighbors,mean_grid)
# plt.errorbar(neighbors,mean_grid,std_grid)

# ax.set_xticklabels(neighbors)
# ax.tick_params(axis='both',labelsize=15)
# # ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# # ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
# plt.xlabel('k_neighbors',size=20,fontweight='bold')
# plt.ylabel('Accuracy',size=20,fontweight='bold')
# plt.show()



/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8181818181818182 {'kernel': 'sigmoid', 'C': 0.1}
[0.80165289 0.73553719 0.81404959 0.81818182 0.7892562  0.74380165
 0.80991736 0.79752066 0.78512397 0.75619835 0.80991736 0.80165289
 0.78099174 0.77272727 0.80991736 0.80991736 0.7892562  0.79338843
 0.81404959 0.80578512 0.78099174 0.7768595  0.81818182 0.80578512
 0.78099174 0.78099174 0.80991736 0.7892562  0.78512397 0.7892562
 0.80165289 0.80165289 0.78099174 0.79338843 0.7892562  0.78512397
 0.78099174 0.7892562  0.78512397 0.78512397 0.78099174 0.79338843
 0.7892562  0.7768595  0.78099174 0.79338843 0.79338843 0.79752066
 0.78099174 0.79338843 0.79338843 0.79338843 0.78512397 0.7892562
 0.79338843 0.78512397 0.78512397 0.78512397 0.79338843 0.76446281
 0.78512397 0.78512397 0.79752066 0.77272727 0.78512397 0.78099174
 0.79752066 0.77272727 0.78512397 0.7768595  0.80165289 0.75619835
 0.78512397 0.7768595  0.80165289 0.77272727 0.78512397 0.76446281
 0.80991736 0.78512397 0.78512397 0.76859504 0.80991736 0.76859504
 0.78512397 0

In [40]:
#FINALIZE the Model

model = SVC(C=0.1,kernel='sigmoid')
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model.fit(rescaledX,y_train)
rescaled_ValidationX = scaler.transform(X_validationset)
result = model.predict(rescaled_ValidationX)
print "Accuracy of SVC (C=0.1,kernel = 'sigmoid') is: ", accuracy_score(y_validationset,result)
print ""
print "The Confusion-Matrix is: ", confusion_matrix(y_validationset,result)
print ""
print "The Classification-Report is: ", classification_report(y_validationset,result)

Accuracy of SVC (C=0.1,kernel = 'sigmoid') is:  0.8032786885245902

The Confusion-Matrix is:  [[28  6]
 [ 6 21]]

The Classification-Report is:                precision    recall  f1-score   support

         0.0       0.82      0.82      0.82        34
         1.0       0.78      0.78      0.78        27

   micro avg       0.80      0.80      0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.80      0.80      0.80        61

